In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install chromadb

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
# # Specify the dataset name and the column containing the content
# # dataset_name = "DebasishDhal99/RAG_Dataset"
# dataset_name = "databricks/databricks-dolly-15k"

# page_content_column = "context"  # or any other column you're interested in

# # Create a loader instance
# loader = HuggingFaceDatasetLoader(dataset_name, page_content_column,)

# # Load the data
# data = loader.load()

# # Display the first 2 entries
# data=data[:100]

# --------------------------------------------------------------------------------------------------------------------------------------------------------
#Specify the dataset name and the column containing the content
dataset_name = "HydraIndicLM/Hindi_Train_ClosedDomainQA"

# dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context" # or any other column you're interested in
from datasets import load_dataset
dataset= load_dataset(dataset_name, page_content_column)
train_dataset = dataset["train"]

context_array = train_dataset['context']
question_array = train_dataset['question']
answer_array = train_dataset['answer']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from langchain.docstore.document import Document

# Create a list to store Document instances
data = []

# Create a Document instance for each set of values
for i in range(len(context_array)):
    document = Document(
        page_content=context_array[i],
        metadata={'instruction': question_array[i], 'response': answer_array[i], }
    )
    data.append(document)

In [ ]:
len(data)
print(type(data))

<class 'list'>


In [ ]:
data=data[:1000]

In [ ]:

# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)
len(docs)

3378

In [ ]:
docs[11]

Document(page_content='जिनकी सभी ने प्रशंसा की विशेषकर भारत के युवाओं के वे प्रशंसनीय नेता थे जिन्हें वे जनता का राष्ट्रपति बुलाते थे। "बांग्लादेश की प्रधानमंत्री शेख हसीना ने उनकी व्याख्या करते हुए कहा, "एक महान राजनेता प्रशंसित वैज्ञानिक और दक्षिण एशिया के युवा पीढ़ी के लिए प्रेरणा स्रोत के संयोग" उन्होंने कलाम की मृत्यु को "भारत के लिए अपूरणीय क्षति से भी परे बताया। " उन्होंने यह भी कहा कि भारत के सबसे प्रसिद्ध बेटे, पूर्व राष्ट्रपति के निधन पर हमें गहरा झटका लगा है। ए॰पी॰जे॰ अब्दुल कलाम अपने समय के सबसे महान ज्ञानियों में से एक थे। वह बांग्लादेश में भी बहुत सम्मानित थे। उनकी विज्ञान और प्रौद्योगिकी के क्षेत्र में भारत की वृद्धि करने के लिए अमूल्य योगदान के लिए वे सभी के द्वारा हमेशा याद किये जायेंगे। वे दक्षिण एशिया की युवा पीढ़ी के लिए प्रेरणा का स्रोत थे जो उनके सपनों को पंख देते थे। " बांग्लादेश नेशनलिस्ट पार्टी की प्रमुख खालिदा जिया ने कहा, "एक परमाणु वैज्ञानिक के रूप में, उन्होंने लोगों के कल्याण में स्वयं को समर्पित किया। "अफगानिस्तान के राष्ट्रपति अशरफ गनी, ने कलाम को, "लाखो

In [ ]:
# docs[18501]

TEXt Embedding

In [ ]:
# # Define the path to the pre-trained model you want to use
# modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# # Create a dictionary with model configuration options, specifying to use the CPU for computations
# model_kwargs = {'device':'cuda'}

# # Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
# encode_kwargs = {'normalize_embeddings': True}

# # Initialize an instance of HuggingFaceEmbeddings with the specified parameters
# embeddings = HuggingFaceEmbeddings(
#     model_name=modelPath,     # Provide the pre-trained model's path
#     model_kwargs=model_kwargs, # Pass the model configuration options
#     encode_kwargs=encode_kwargs # Pass the encoding options
# )

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
encode_kwargs = {'normalize_embeddings': False} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

.gitattributes:   0%|          | 0.00/968 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

### Using Chroma & Parent-doc-retriever

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=embeddings,  #OpenAIEmbeddings()
)

# The storage layer for the parent documents
store = InMemoryStore()

full_doc_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=text_splitter,
)

In [ ]:
full_doc_retriever.add_documents(docs)

In [ ]:
retrieved_docs = full_doc_retriever.get_relevant_documents("भारतीय संविधान कब लागू हुआ था ?")

In [ ]:
retrieved_docs

[Document(page_content="'पहला और सबसे महत्वपूर्ण सामाजिक दस्तावेज' के रूप में आम्बेडकर द्वारा तैयार भारतीय संविधान का वर्णन किया। 'भारत के अधिकांश संवैधानिक प्रावधान या तो सामाजिक क्रांति के उद्देश्य को आगे बढ़ाने या इसकी उपलब्धि के लिए जरूरी स्थितियों की स्थापना करके इस क्रांति को बढ़ावा देने के प्रयास में सीधे पहुँचे हैं। 'आम्बेडकर द्वारा तैयार किए गए संविधान के पाठ में व्यक्तिगत नागरिकों के लिए नागरिक स्वतंत्रता की एक विस्तृत श्रृंखला के लिए संवैधानिक गारंटी और सुरक्षा प्रदान की गई है, जिसमें धर्म की आजादी, छुआछूत को खत्म करना, और भेदभाव के सभी रूपों का उल्लंघन करना शामिल है। आम्बेडकर ने महिलाओं के लिए व्यापक आर्थिक और सामाजिक अधिकारों के लिए तर्क दिया, और अनुसूचित जातियों (एससी) और अनुसूचित जनजातियों (एसटी) और अन्य पिछड़ा वर्ग (ओबीसी) के सदस्यों के लिए नागरिक सेवाओं, स्कूलों और कॉलेजों में नौकरियों के आरक्षण की व्यवस्था शुरू करने के लिए असेंबली का समर्थन जीता, जो कि सकारात्मक कार्रवाई थी। भारत के सांसदों ने इन उपायों के माध्यम से भारत की निराशाजनक कक्षाओं के लिए सामाजिक-आर्थिक असमा

In [ ]:
sub_docs = vectorstore.similarity_search("भारतीय संविधान कब लागू हुआ था ?",k=2)

In [ ]:
sub_docs

[Document(page_content="'पहला और सबसे महत्वपूर्ण सामाजिक दस्तावेज' के रूप में आम्बेडकर द्वारा तैयार भारतीय संविधान का वर्णन किया। 'भारत के अधिकांश संवैधानिक प्रावधान या तो सामाजिक क्रांति के उद्देश्य को आगे बढ़ाने या इसकी उपलब्धि के लिए जरूरी स्थितियों की स्थापना करके इस क्रांति को बढ़ावा देने के प्रयास में सीधे पहुँचे हैं। 'आम्बेडकर द्वारा तैयार किए गए संविधान के पाठ में व्यक्तिगत नागरिकों के लिए नागरिक स्वतंत्रता की एक विस्तृत श्रृंखला के लिए संवैधानिक गारंटी और सुरक्षा प्रदान की गई है, जिसमें धर्म की आजादी, छुआछूत को खत्म करना, और भेदभाव के सभी रूपों का उल्लंघन करना शामिल है। आम्बेडकर ने महिलाओं के लिए व्यापक आर्थिक और सामाजिक अधिकारों के लिए तर्क दिया, और अनुसूचित जातियों (एससी) और अनुसूचित जनजातियों (एसटी) और अन्य पिछड़ा वर्ग (ओबीसी) के सदस्यों के लिए नागरिक सेवाओं, स्कूलों और कॉलेजों में नौकरियों के आरक्षण की व्यवस्था शुरू करने के लिए असेंबली का समर्थन जीता, जो कि सकारात्मक कार्रवाई थी। भारत के सांसदों ने इन उपायों के माध्यम से भारत की निराशाजनक कक्षाओं के लिए सामाजिक-आर्थिक असमा

### Using FAISS

In [ ]:
text = "संवेदना ."
query_result = embeddings.embed_query(text)
query_result[:10]

[0.04707549512386322,
 0.14438828825950623,
 -0.016146859154105186,
 0.14013734459877014,
 -0.21326765418052673,
 -0.15556108951568604,
 0.07024840265512466,
 0.12703856825828552,
 0.046366576105356216,
 -0.11819376796483994]

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
db

In [ ]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("जब ए.पी.जे. अब्दुल कलाम की मृत्यु हुई थी उस समय श्रीलंका के राष्ट्रपति कौन थे ?")
print(docs[0].page_content)

राणा उदयसिंह केे दूसरी रानी धीरबाई जिसे राज्य के इतिहास में रानी भटियाणी के नाम से जाना जाता है, यह अपने पुत्र कुंवर जगमाल को मेवाड़ का उत्तराधिकारी बनाना चाहती थी | प्रताप केे उत्तराधिकारी होने पर इसकेे विरोध स्वरूप जगमाल अकबर केे खेमे में चला जाता है। महाराणा प्रताप का प्रथम राज्याभिषेक मेंं 28 फरवरी, 1572 में गोगुन्दा में हुआ था, लेकिन विधि विधानस्वरूप राणा प्रताप का द्वितीय राज्याभिषेक 1572 ई. में ही कुुंभलगढ़़ दुुर्ग में हुआ, दुसरे राज्याभिषेक में जोधपुर का राठौड़ शासक राव चन्द्रसेेन भी उपस्थित थे। राणा प्रताप ने अपने जीवन में कुल ११ शादियाँ की थी यह युद्ध 18 जून 1576 ईस्वी में मेवाड़ तथा मुगलों के मध्य हुआ था। इस युद्ध में मेवाड़ की सेना का नेतृत्व महाराणा प्रताप ने किया था। भील सेना के सरदार राणा पूंजा भील थे। इस युद्ध में महाराणा प्रताप की तरफ से लड़ने वाले एकमात्र मुस्लिम सरदार थे- हकीम खाँ सूरी। लड़ाई का स्थल राजस्थान के गोगुन्दा के पास हल्दीघाटी में एक संकरा पहाड़ी दर्रा था। महाराणा प्रताप ने लगभग 3,000 घुड़सवारों और 400 भील धनुर्धारियों के बल को मैदान में उतारा। मुगलों का


In [ ]:
question = '''जब ए.पी.जे. अब्दुल कलाम की मृत्यु हुई थी उस समय श्रीलंका के राष्ट्रपति कौन थे ?'''
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

राणा उदयसिंह केे दूसरी रानी धीरबाई जिसे राज्य के इतिहास में रानी भटियाणी के नाम से जाना जाता है, यह अपने पुत्र कुंवर जगमाल को मेवाड़ का उत्तराधिकारी बनाना चाहती थी | प्रताप केे उत्तराधिकारी होने पर इसकेे विरोध स्वरूप जगमाल अकबर केे खेमे में चला जाता है। महाराणा प्रताप का प्रथम राज्याभिषेक मेंं 28 फरवरी, 1572 में गोगुन्दा में हुआ था, लेकिन विधि विधानस्वरूप राणा प्रताप का द्वितीय राज्याभिषेक 1572 ई. में ही कुुंभलगढ़़ दुुर्ग में हुआ, दुसरे राज्याभिषेक में जोधपुर का राठौड़ शासक राव चन्द्रसेेन भी उपस्थित थे। राणा प्रताप ने अपने जीवन में कुल ११ शादियाँ की थी यह युद्ध 18 जून 1576 ईस्वी में मेवाड़ तथा मुगलों के मध्य हुआ था। इस युद्ध में मेवाड़ की सेना का नेतृत्व महाराणा प्रताप ने किया था। भील सेना के सरदार राणा पूंजा भील थे। इस युद्ध में महाराणा प्रताप की तरफ से लड़ने वाले एकमात्र मुस्लिम सरदार थे- हकीम खाँ सूरी। लड़ाई का स्थल राजस्थान के गोगुन्दा के पास हल्दीघाटी में एक संकरा पहाड़ी दर्रा था। महाराणा प्रताप ने लगभग 3,000 घुड़सवारों और 400 भील धनुर्धारियों के बल को मैदान में उतारा। मुगलों का


In [ ]:
searchDocs

[Document(page_content='राणा उदयसिंह केे दूसरी रानी धीरबाई जिसे राज्य के इतिहास में रानी भटियाणी के नाम से जाना जाता है, यह अपने पुत्र कुंवर जगमाल को मेवाड़ का उत्तराधिकारी बनाना चाहती थी | प्रताप केे उत्तराधिकारी होने पर इसकेे विरोध स्वरूप जगमाल अकबर केे खेमे में चला जाता है। महाराणा प्रताप का प्रथम राज्याभिषेक मेंं 28 फरवरी, 1572 में गोगुन्दा में हुआ था, लेकिन विधि विधानस्वरूप राणा प्रताप का द्वितीय राज्याभिषेक 1572 ई. में ही कुुंभलगढ़़ दुुर्ग में हुआ, दुसरे राज्याभिषेक में जोधपुर का राठौड़ शासक राव चन्द्रसेेन भी उपस्थित थे। राणा प्रताप ने अपने जीवन में कुल ११ शादियाँ की थी यह युद्ध 18 जून 1576 ईस्वी में मेवाड़ तथा मुगलों के मध्य हुआ था। इस युद्ध में मेवाड़ की सेना का नेतृत्व महाराणा प्रताप ने किया था। भील सेना के सरदार राणा पूंजा भील थे। इस युद्ध में महाराणा प्रताप की तरफ से लड़ने वाले एकमात्र मुस्लिम सरदार थे- हकीम खाँ सूरी। लड़ाई का स्थल राजस्थान के गोगुन्दा के पास हल्दीघाटी में एक संकरा पहाड़ी दर्रा था। महाराणा प्रताप ने लगभग 3,000 घुड़सवारों और 400 भील धनुर्धारियों के बल को मैदा

### LLM

In [ ]:

# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("OdiaGenAI/mistral_hindi_7b_base_v1")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
model = AutoModelForCausalLM.from_pretrained("OdiaGenAI/mistral_hindi_7b_base_v1")

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

pytorch_model-00004-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00005-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

In [ ]:
# Specify the model name you want to use
model_name = "OdiaGenAI/mistral_hindi_7b_base_v1"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
text_generation = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=text_generation,
    model_kwargs={"temperature": 0.7, "max_length": 512},)

In [ ]:
# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)